In [19]:
import random

CUSTOMERS = [(0, 0), (1, 5), (2, 2), (3, 8), (5, 5), (6, 1), (8, 3)]

POPULATION_SIZE = 4  # Smaller population for demonstration
GENERATIONS = 2
VEHICLES = 3

def create_individual():
    """
    Creates a random individual for a multi-vehicle TSP/VRP.
    Representation: List of lists, one list per vehicle containing assigned customers.
    Allows uneven distribution of customers.
    """
    customers = list(range(1, len(CUSTOMERS)))  # exclude depot (index 0)
    random.shuffle(customers)

    individual = [[] for _ in range(VEHICLES)]

    for cust in customers:
        chosen_vehicle = random.randint(0, VEHICLES - 1)  # assign randomly
        individual[chosen_vehicle].append(cust)

    return individual

In [20]:
# Testing creation of individuals
INDIVIDUALS = []
for i in range(POPULATION_SIZE):
  INDIVIDUALS.append(create_individual())

INDIVIDUALS

[[[3, 1, 2, 4], [5], [6]],
 [[], [3, 2, 4], [6, 5, 1]],
 [[], [6, 3, 2], [1, 4, 5]],
 [[2, 1], [3, 6], [4, 5]]]

In [21]:
def fitness(individual):
    individual = decode_individual(individual) #adding depot for start and end
    total_distance = 0

    for route in individual:  # go through each vehicle's route
        if len(route) > 1:  # if vehicle visits more than one customer
            for i in range(1, len(route)):
                x1, y1 = CUSTOMERS[route[i-1]]
                x2, y2 = CUSTOMERS[route[i]]
                total_distance += ((x2 - x1)**2 + (y2 - y1)**2)**0.5

    return 1 / total_distance if total_distance > 0 else 0


def decode_individual(individual):
    return [[0] + route + [0] for route in individual]

In [22]:
FITNESSES = []
for individual in INDIVIDUALS:
  FITNESSES.append(fitness(individual))

FITNESSES

[0.017895786977625883,
 0.020485485774972813,
 0.02281482519881399,
 0.019038286454653656]

In [23]:
def tournament_selection(population, fitnesses, k=3):
    # pick k random indices
    participants = random.sample(range(len(population)), k)
    print("participants: ", participants)
    
    # find the best among them
    best_idx = participants[0]
    for idx in participants[1:]:
        if fitnesses[idx] > fitnesses[best_idx]:
            best_idx = idx
    
    return population[best_idx]

In [24]:
selected = tournament_selection(INDIVIDUALS, FITNESSES)
print("Selected individual:", selected)

participants:  [0, 3, 2]
Selected individual: [[], [6, 3, 2], [1, 4, 5]]


In [25]:
def swap_mutation(individual):
    vehicle = random.choice([v for v in individual if len(v) > 1])
    i, j = random.sample(range(len(vehicle)), 2)
    vehicle[i], vehicle[j] = vehicle[j], vehicle[i]
    print("swap")
    return individual


def relocation_mutation(individual):
    # pick a non-empty vehicle as source
    from_vehicle = random.choice([v for v in individual if v])
    cust = from_vehicle.pop(random.randrange(len(from_vehicle)))

    # pick a different vehicle as target
    candidates = [v for v in individual if v is not from_vehicle]
    to_vehicle = random.choice(candidates)

    # insert at random position
    insert_pos = random.randint(0, len(to_vehicle))
    to_vehicle.insert(insert_pos, cust)

    print("relocation")
    return individual


def mutate(individual):
    chance = random.random()
    if chance <= 1:  # 10% mutation rate for demonstration
      if chance < 0.5: # 50% of each mutation type
        relocation_mutation(individual)
      else: swap_mutation(individual)
    return individual

In [26]:
for individual in INDIVIDUALS:
  mutate(individual)

INDIVIDUALS

relocation
relocation
relocation
swap


[[[3, 1, 2, 6, 4], [5], []],
 [[], [1, 3, 2, 4], [6, 5]],
 [[4], [6, 3, 2], [1, 5]],
 [[2, 1], [3, 6], [5, 4]]]